In [ ]:
# -----------------------------------------------------------------------------
# 📘 Notebook: 02_data_cleaning_and_summary.ipynb
#
# Purpose:
#   Prepare a clean, high-quality summary dataset of all running sessions.
#   This notebook filters out non-running or corrupted data, aggregates
#   individual GPS records into one-row-per-run summaries, and enriches them
#   with temporal and quality information.
#
# Context:
#   - Follows Stage 1 (EDA & Data Quality Audit)
#   - Produces a ready-to-ingest dataset for Neo4j and ML analysis
#
# Main steps:
#   1. Load raw processed data (strava_runs.parquet)
#   2. Apply cleaning and filtering rules based on the data quality manifest
#   3. Aggregate per run (distance, pace, cadence, heart rate, elevation)
#   4. Add date/time and quality metrics
#   5. Save the final summary to:
#        data/strava/processed/run_summary_cleaned.{csv,parquet}
#
# Input:
#   data/strava/processed/strava_runs.parquet
#   data/strava/processed/data_quality_manifest.csv
#
# Output:
#   data/strava/processed/run_summary_cleaned.parquet
#   data/strava/processed/run_summary_cleaned.csv
#
# Next step:
#   Stage 3 → Neo4j graph ingestion and visualization
# -----------------------------------------------------------------------------


In [1]:
# --- Stage 2: Cleaning & Per-Run Summary ---------------------------------

import pandas as pd
import numpy as np
from pathlib import Path

raw_path = Path("../data/strava/processed/strava_runs.parquet")
df = pd.read_parquet(raw_path)
print(f"Loaded {len(df):,} rows")


Loaded 600,047 rows


In [2]:
# --- 2.0 Create or refresh data quality manifest --------------------------
from pathlib import Path
import pandas as pd

processed_dir = Path("data/strava/processed")
manifest_path = processed_dir / "data_quality_manifest.csv"

# Whether to force regeneration
force_recreate_manifest = True
processed_dir.mkdir(parents=True, exist_ok=True)

def classify_column(col: str, missing_pct: float, nunique: int) -> str:
    """Heuristic classification of columns."""
    col_lower = col.lower()
    if col_lower.startswith("unknown") or "unnamed" in col_lower:
        return "drop"
    if missing_pct > 70:
        return "review"
    if missing_pct > 50:
        return "optional"
    if nunique <= 1:
        return "drop"  # constant or empty
    if col_lower in [
        "distance", "distance_km", "pace_min_per_km", "timestamp",
        "speed", "cadence", "altitude", "heart_rate", "run_id",
        "lat", "lon", "position_lat", "position_long",
    ]:
        return "core"
    return "optional"

if not manifest_path.exists() or force_recreate_manifest:
    print("🔄 Creating or refreshing data quality manifest...")

    # Compute stats
    stats = pd.DataFrame({
        "column": df.columns,
        "dtype": [str(df[c].dtype) for c in df.columns],
        "non_null_count": df.notna().sum().values,
        "missing_pct": (df.isna().mean().values * 100).round(1),
        "nunique": df.nunique().values,
    })

    stats["category"] = [
        classify_column(c, m, n)
        for c, m, n in zip(stats["column"], stats["missing_pct"], stats["nunique"])
    ]

    stats = stats.sort_values(["category", "missing_pct"], ascending=[True, False])

    stats.to_csv(manifest_path, index=False)
    print(f"✅ Manifest refreshed with {len(stats)} columns at: {manifest_path.resolve()}")
    print(stats["category"].value_counts().to_string())
else:
    print(f"✅ Manifest already exists and not overwritten: {manifest_path.resolve()}")

# Preview top of manifest
pd.read_csv(manifest_path).head(10)


🔄 Creating or refreshing data quality manifest...
✅ Manifest refreshed with 26 columns at: C:\Users\ITSMARTSOLUTIONS\Documents\Python Scripts\notebook-llm-lab\notebooks\data\strava\processed\data_quality_manifest.csv
category
core        12
optional    11
drop         2
review       1


,column,dtype,non_null_count,missing_pct,nunique,category
0,pace_min_per_km,float64,428221,28.6,2110,core
1,cadence,float64,430449,28.3,119,core
2,speed,float64,432633,27.9,2111,core
3,position_lat,float64,561198,6.5,429110,core
4,position_long,float64,561198,6.5,475613,core
5,lat,float64,561198,6.5,429110,core
6,lon,float64,561198,6.5,475613,core
7,altitude,float64,579695,3.4,3371,core
8,timestamp,datetime64[ns],600047,0.0,597700,core
9,distance,float64,600047,0.0,471475,core


In [3]:
# --- 2.1 Drop unknown or low-quality columns ------------------------------
manifest = pd.read_csv("../data/strava/processed/data_quality_manifest.csv", index_col="column")
keep_cols = manifest.query("category != 'drop'").index.tolist()
df = df[keep_cols]
print(f"Keeping {len(keep_cols)} columns based on manifest")


Keeping 15 columns based on manifest


In [4]:
# --- 2.2 Apply activity-level filters ------------------------------------
before = len(df)
df = df.dropna(subset=["distance_km", "pace_min_per_km"])
df = df[df["distance_km"] >= 0.5]
df = df[(df["pace_min_per_km"] >= 2) & (df["pace_min_per_km"] <= 15)]
df = df[~((df["cadence"] == 0) & (df["speed"] == 0))]
print(f"Removed {before - len(df):,} noisy rows")


Removed 208,351 noisy rows


In [5]:
# --- 2.3 Aggregate per-run summary (robust to missing HR) ----------------
agg_spec = dict(
    records=("timestamp", "count"),
    start_time=("timestamp", "min"),
    end_time=("timestamp", "max"),
    total_distance_km=("distance_km", "max"),
    avg_pace=("pace_min_per_km", "mean"),
    avg_speed=("speed", "mean"),
    avg_cadence=("cadence", "mean"),
    elevation_gain=("altitude", lambda s: s.max() - s.min()),
)

if "heart_rate" in df.columns:
    agg_spec["avg_hr"] = ("heart_rate", "mean")

summary = df.groupby("run_id").agg(**agg_spec).reset_index()


In [6]:
# --- 2.4 Add date context -------------------------------------------------
summary["date"] = pd.to_datetime(summary["start_time"]).dt.date
summary["weekday"] = pd.to_datetime(summary["start_time"]).dt.day_name()
summary["month"] = pd.to_datetime(summary["start_time"]).dt.to_period("M")



In [7]:
# --- 2.5 Run-level quality metrics ---------------------------------------

# per-run % missing values
missing_pct = (
    df.isna()
      .groupby(df["run_id"])
      .mean()
      .mean(axis=1) * 100
)
summary = summary.merge(missing_pct.rename("missing_pct"), on="run_id", how="left")

# per-run duration in minutes
summary["duration_min"] = (
    (pd.to_datetime(summary["end_time"]) - pd.to_datetime(summary["start_time"]))
    .dt.total_seconds() / 60
)


In [8]:
# --- 2.6 Save results -----------------------------------------------------
out_path = Path("../data/strava/processed/run_summary_cleaned.parquet")
summary.to_parquet(out_path, index=False)
summary.to_csv(out_path.with_suffix(".csv"), index=False)
print(f"✅ Saved {len(summary):,} clean runs → {out_path}")

summary.head()

✅ Saved 697 clean runs → ..\data\strava\processed\run_summary_cleaned.parquet


,run_id,records,start_time,end_time,total_distance_km,avg_pace,avg_speed,avg_cadence,elevation_gain,date,weekday,month,missing_pct,duration_min
0,10011163869.fit,595,2023-06-25 14:30:11,2023-06-25 15:24:00,12.06647,4.482086,3.767400,83.732773,21.0,2023-06-25,Sunday,2023-06,0.000000,53.816667
1,10047123126.fit,643,2023-07-01 08:18:38,2023-07-01 09:22:26,8.35794,5.641040,3.130320,81.850700,295.4,2023-07-01,Saturday,2023-07,0.000000,63.800000
2,10049122762.fit,339,2023-07-01 12:36:08,2023-07-01 14:02:36,3.70666,13.431392,1.254997,46.209440,76.6,2023-07-01,Saturday,2023-07,0.157325,86.466667
3,10075256002.fit,681,2023-07-05 12:52:57,2023-07-05 13:57:53,13.20205,4.812680,3.542367,82.776799,24.4,2023-07-05,Wednesday,2023-07,0.000000,64.933333
4,10109043290.fit,537,2023-07-10 15:15:55,2023-07-10 16:00:05,11.00990,4.286735,3.929385,84.372439,24.2,2023-07-10,Monday,2023-07,0.000000,44.166667
